### CREATE TEST QUERIES

**jira**: https://jira.bltelecoms.net/browse/SWITZERLAND-428

**ref**:  
* https://www.tutorialspoint.com/sqlite/sqlite_python.htm  
* https://www.r-bloggers.com/turning-data-into-awesome-with-sqldf-and-pandasql/  
* https://pypi.python.org/pypi/pandasql

In [1]:
import pandas as pd
import sqlite3

from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
conn = sqlite3.connect("etl.db")

In [44]:
companyTableDF = pd.read_sql_query("select * from COMPANY;", conn)
vehiclesTableDF = pd.read_sql_query("select * from VEHICLES;", conn)

In [78]:
print(companyTableDF)

   ID   NAME  AGE     ADDRESS    SALARY
0   1   Paul   32  California   20000.0
1   2  Allen   25       Texas   15000.0
2   3  Teddy   23      Norway   20000.0
3   4   Mark   25  Rich-Mond    65000.0
4   5   Dave   45  California   20010.0
5   6    Bob   25       Texas   16000.0
6   7   Mike   23      Norway   21000.0
7   8   Mark   55    Chicago   150000.0
8   9   Mark   45     Boston   800000.0
9  10   Mark   65    Florida   120000.0


In [79]:
print(vehiclesTableDF)

   ID    MAKE  CYLINDERS TRANSMISSION
0   1    Ford          6         Auto
1   2      VW          6       Manual
2   3  Peugot          6         Auto
3   4    Fiat          4         Auto
4   5  Toyota          4       Manual
5   6  Toyota          4         Auto
6   7    Ford          6       Manual
7   8  Toyota          4         Auto
8   9     BMW          6         Auto
9  10     BMW          6       Manual


In [10]:
# companyTableDF  COMPANY:  ID,NAME,AGE,ADDRESS,SALARY
# vehiclesTableDF VEHICLES: ID,MAKE,CYLINDERS,TRANSMISSION

q  = """
SELECT * 
FROM companyTableDF c 
WHERE c.NAME IN ('Mike', 'Mark')
;
"""

In [11]:
qDF = pysqldf(q)
# select iso_country, type, count(*) from airports group by iso_country, type order by iso_country, type

In [77]:
print(qDF)

   ID  NAME  AGE     ADDRESS    SALARY
0   4  Mark   25  Rich-Mond    65000.0
1   7  Mike   23      Norway   21000.0
2   8  Mark   55    Chicago   150000.0
3   9  Mark   45     Boston   800000.0
4  10  Mark   65    Florida   120000.0


In [41]:
q1  = """
SELECT c.ADDRESS
      ,SUM(SALARY) AS totalSalary
      ,COUNT(SALARY) AS count
      ,AVG(SALARY) AS averageSalary 
FROM companyTableDF c 
GROUP BY ADDRESS
ORDER BY ADDRESS DESC
;
"""

In [42]:
q1DF = pysqldf(q1)

In [76]:
print(q1DF)

      ADDRESS  totalSalary  count  averageSalary
0       Texas      31000.0      2        15500.0
1  Rich-Mond       65000.0      1        65000.0
2      Norway      41000.0      2        20500.0
3    Florida      120000.0      1       120000.0
4    Chicago      150000.0      1       150000.0
5  California      40010.0      2        20005.0
6     Boston      800000.0      1       800000.0


In [46]:
q2  = """
        SELECT 
               c.*
              ,v.* 
        FROM companyTableDF c
        LEFT JOIN vehiclesTableDF v ON c.ID = v.ID 
        WHERE v.MAKE = 'BMW'
      ;
"""

In [47]:
q2DF = pysqldf(q2)

In [75]:
print(q2DF)

   ID  NAME  AGE   ADDRESS    SALARY  ID MAKE  CYLINDERS TRANSMISSION
0   9  Mark   45   Boston   800000.0   9  BMW          6         Auto
1  10  Mark   65  Florida   120000.0  10  BMW          6       Manual


In [71]:
# companyTableDF  COMPANY:  ID,NAME,AGE,ADDRESS,SALARY
# vehiclesTableDF  VEHICLES: ID,MAKE,CYLINDERS,TRANSMISSION

q3  = """ SELECT 
            c.ADDRESS
            ,v.MAKE
            ,COUNT(c.SALARY) AS count
      FROM companyTableDF c
      LEFT JOIN vehiclesTableDF v ON c.ID = v.ID WHERE c.SALARY > 30000
      GROUP BY c.ADDRESS,v.MAKE
;
"""

In [72]:
q3DF = pysqldf(q3)

In [ ]:
print("\nquery results\n")

In [74]:
print(q3DF)

      ADDRESS    MAKE  count
0     Boston      BMW      1
1    Chicago   Toyota      1
2    Florida      BMW      1
3  Rich-Mond     Fiat      1


In [ ]:
conn.close();